In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings
import sys
import matplotlib.pyplot as plt
import json
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from keras import layers
import math
from sklearn.metrics import mean_squared_error
from datetime import datetime
import datetime as dt
from pylab import rcParams
from numpy import array


In [9]:
# Read Data from CSV into Dataframe
df = pd.read_csv('/content/drive/MyDrive/JPM/jphomevalue/data/HousePriceDataStPete.csv')
df.head()

,DATE,ATNHPIUS45300Q,"5725 80th St N UNIT 406, Saint Petersburg, FL 33709","5286 81st St N APT 21, Saint Petersburg, FL 33709","5257 81st Ln N APT 23, Saint Petersburg, FL 33709","115 112th Ave NE APT 1027, Saint Petersburg, FL 33716","3135 34th Ave N, Saint Petersburg, FL 33713","968 Melrose Ave S, Saint Petersburg, FL 33705","5246 4th Ave N, Saint Petersburg, FL 33710","2726 16th Ave N, Saint Petersburg, FL 33713",...,"4710 21st St N, Saint Petersburg, FL 33714","718 4th Ave N, Saint Petersburg, FL 33701","648 58th Ave NE, Saint Petersburg, FL 33703","1717 Massachusetts Ave NE, Saint Petersburg, FL 33703","2010 Seminole Blvd S, Saint Petersburg, FL 33705","3089 15th Ave S, Saint Petersburg, FL 33712","355 Brightwaters Blvd NE, Saint Petersburg, FL 33704","3190 69th Ave N, Saint Petersburg, FL 33702","5201 66th St N, St Petersburg, FL 33709","3430 19th Ave S, Saint Petersburg, FL 33711"
0,2002-01-31,152.713333,37262.05333,27793.82667,26419.40667,106593.9067,72538.83333,34971.35333,57572.92667,44248.68833,...,27488.40000,432636.8733,61390.76000,126599.3533,13133.34667,31917.08667,1050973.160,44592.29333,408966.3067,34665.92667
1,2002-02-28,153.995111,37574.80711,28027.11022,26641.15422,107488.5876,73147.67778,35264.88044,58056.15689,44620.08344,...,27719.12000,436268.1498,61906.03467,127661.9471,13243.57956,32184.97822,1059794.355,44966.57244,412398.9076,34956.89022
2,2002-03-31,155.414222,37921.07022,28285.38844,26886.66044,108479.1271,73821.75556,35589.85689,58591.16178,45031.27089,...,27974.56000,440288.4916,62476.51733,128838.3902,13365.62311,32481.57244,1069560.677,45380.95289,416199.2871,35279.02844
3,2002-04-30,156.763407,38250.27121,28530.94000,27120.06934,109420.8578,74462.61813,35898.82011,59099.80429,45422.19706,...,28217.41319,444110.7309,63018.88945,129956.8641,13481.65297,32763.55198,1078845.764,45774.91473,419812.4029,35585.29330
4,2002-05-31,158.156703,38590.23560,28784.52000,27361.10967,110393.3789,75124.43407,36217.88505,59625.07714,45825.90478,...,28468.20659,448057.9404,63578.99473,131111.9070,13601.47648,33054.75099,1088434.432,46181.75736,423543.6514,35901.57165


In [10]:
# ********* PRIOR INITIALIZATIONS **************************
# *************** RUN ONCE *********************************

# Load the model to be used
model = keras.models.load_model('/content/drive/MyDrive/JPM/jphomevalue/specific_model_training/lstm-model-st.pete-24-3.h5')

# Create a skeleton dataframe that will be initialized with Daily Shiller Data
shiller_filePath = "/content/drive/MyDrive/JPM/jphomevalue/data/StPeteShiller12monthsPredicted.csv"
shiller_data = pd.read_csv(shiller_filePath, header = 0)

m_df = shiller_data.copy()

# Scaler
scaler = MinMaxScaler(feature_range=(0,1))


In [11]:
# auto_prediction() function

def auto_prediction(house_data, house_address): 
  
  # Normalize and Reshape the house data passed
  house_data = scaler.fit_transform(np.array(house_data).reshape(-1,1))

  # Use the last 24 entries of the house_data for prediction
  prediction_input = house_data[int(house_data.shape[0])-24:int(selective_df.shape[0]),:].reshape(1,-1)

  # Call the prediction generator function to return the new predictions
  new_predictions = prediction_generator(prediction_input)
  
  return new_predictions



In [12]:
# prediction_generator() function

def prediction_generator(pred_input_data):

    pred_input = pred_input_data
    # Add pred_input to temp_input
    temp_input = []
    temp_input=list(pred_input)
    temp_input=temp_input[0].tolist()

    # Predict timesteps in the future
    list_output=[]
    n_steps=24
    k=0

    while(k<12):
        
        if(len(temp_input)>24):
            print('Entering main condition')
            #print(temp_input)
            pred_input=np.array(temp_input[1:]) # Shift one to the right
            print("{} month input {}".format(k,pred_input))
            pred_input=pred_input.reshape(1,-1)
            pred_input = pred_input.reshape(1, n_steps, 1)
            print('Pred_input shape is:', pred_input.shape)
            fut_pred = model.predict(pred_input, verbose=0)
            print("{} month output {}".format(k,fut_pred))
            temp_input.extend(fut_pred[0].tolist()) # Add that prediction to the temp input list as that will be used in the next prediction 
            temp_input=temp_input[1:] #Shift the temp_input to remove the first element which was already used in a sequence
            #print(temp_input)
            list_output.extend(fut_pred.tolist())
            k=k+1
        else:
            print('Entering else condition')
            pred_input = pred_input.reshape((1, n_steps,1))
            fut_pred = model.predict(pred_input, verbose=0)
            print('The predicted value is:',fut_pred[0])
            temp_input.extend(fut_pred[0].tolist())
            print(len(temp_input))
            list_output.extend(fut_pred.tolist())
            k=k+1
        

    list_output = np.array(scaler.inverse_transform(list_output))
    returned_output = list_output[:,0]
    return returned_output





In [13]:
# ********* AUTOMATION BEGINS HERE! ************************

# Each column represents a house and it's historical prices
# Isolate a Date|House pair and store it into another df

for i in range(df.shape[1]-1): 
   # Store Date|House Pair and store to selective_df
   selective_df = df.iloc[:, 2+i]

   # Store the current column name ('current house address)
   current_house = df.columns.values[2+i]

   # Call auto_prediction() function to automatically predict new prices and append to a new dataframe
   prediction_data = auto_prediction(selective_df, current_house) 
   #prediction = predictions.tolist()
   print('Returned from auto pred function')
   df_temp = selective_df.tolist()
   df_temp.extend(prediction_data)

   # Start appending to new main dataframe
   m_df[current_house] = df_temp


# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('dataTest.csv')

# Download final CSV file and it is ready to populate to DB
files.download('dataTest.csv')

  


Streaming output truncated to the last 5000 lines.
Entering main condition
8 month input [0.65595307 0.66659111 0.676886   0.6929098  0.70859043 0.72479375
 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525 0.91653199
 0.94480878 0.97285677 1.         1.         0.87939835 0.85364813
 0.80411452 0.74378055 0.67911363 0.61255002 0.54512078 0.47759676]
Pred_input shape is: (1, 24, 1)
8 month output [[0.4106506]]
Entering main condition
9 month input [0.66659111 0.676886   0.6929098  0.70859043 0.72479375 0.75504821
 0.78279787 0.8135207  0.84738495 0.88252525 0.91653199 0.94480878
 0.97285677 1.         1.         0.87939835 0.85364813 0.80411452
 0.74378055 0.67911363 0.61255002 0.54512078 0.47759676 0.41065061]
Pred_input shape is: (1, 24, 1)
9 month output [[0.34478366]]
Entering main condition
10 month input [0.676886   0.6929098  0.70859043 0.72479375 0.75504821 0.78279787
 0.8135207  0.84738495 0.88252525 0.91653199 0.94480878 0.97285677
 1.         1.         0.87939835 0.853

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


4 month output [[0.6791136]]
Entering main condition
5 month input [0.62616484 0.63591151 0.64534377 0.65595307 0.66659111 0.676886
 0.6929098  0.70859043 0.72479375 0.75504821 0.78279787 0.8135207
 0.84738495 0.88252525 0.91653199 0.94480878 0.97285678 1.
 1.         0.87939835 0.85364813 0.80411452 0.74378055 0.67911363]
Pred_input shape is: (1, 24, 1)
5 month output [[0.61255]]
Entering main condition
6 month input [0.63591151 0.64534377 0.65595307 0.66659111 0.676886   0.6929098
 0.70859043 0.72479375 0.75504821 0.78279787 0.8135207  0.84738495
 0.88252525 0.91653199 0.94480878 0.97285678 1.         1.
 0.87939835 0.85364813 0.80411452 0.74378055 0.67911363 0.61255002]
Pred_input shape is: (1, 24, 1)
6 month output [[0.5451208]]
Entering main condition
7 month input [0.64534377 0.65595307 0.66659111 0.676886   0.6929098  0.70859043
 0.72479375 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525
 0.91653199 0.94480878 0.97285678 1.         1.         0.87939835
 0.85364813 0.8041

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


The predicted value is: [0.87939835]
25
Entering main condition
1 month input [0.60082579 0.60631854 0.61141895 0.61687111 0.62616484 0.63591151
 0.64534377 0.65595307 0.66659111 0.676886   0.6929098  0.70859043
 0.72479375 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525
 0.91653199 0.94480878 0.97285678 1.         1.         0.87939835]
Pred_input shape is: (1, 24, 1)
1 month output [[0.8536481]]
Entering main condition
2 month input [0.60631854 0.61141895 0.61687111 0.62616484 0.63591151 0.64534377
 0.65595307 0.66659111 0.676886   0.6929098  0.70859043 0.72479375
 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525 0.91653199
 0.94480878 0.97285678 1.         1.         0.87939835 0.85364813]
Pred_input shape is: (1, 24, 1)
2 month output [[0.8041145]]
Entering main condition
3 month input [0.61141895 0.61687111 0.62616484 0.63591151 0.64534377 0.65595307
 0.66659111 0.676886   0.6929098  0.70859043 0.72479375 0.75504821
 0.78279787 0.8135207  0.84738495 0.88252525 0.91653

...

The predicted value is: [0.87939835]
25
Entering main condition
1 month input [0.60082579 0.60631854 0.61141895 0.61687111 0.62616484 0.63591151
 0.64534377 0.65595307 0.66659111 0.676886   0.6929098  0.70859043
 0.72479375 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525
 0.91653199 0.94480878 0.97285677 1.         1.         0.87939835]
Pred_input shape is: (1, 24, 1)
1 month output [[0.8536481]]
Entering main condition
2 month input [0.60631854 0.61141895 0.61687111 0.62616484 0.63591151 0.64534377
 0.65595307 0.66659111 0.676886   0.6929098  0.70859043 0.72479375
 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525 0.91653199
 0.94480878 0.97285677 1.         1.         0.87939835 0.85364813]
Pred_input shape is: (1, 24, 1)
2 month output [[0.8041145]]
Entering main condition
3 month input [0.61141895 0.61687111 0.62616484 0.63591151 0.64534377 0.65595307
 0.66659111 0.676886   0.6929098  0.70859043 0.72479375 0.75504821
 0.78279787 0.8135207  0.84738495 0.88252525 0.91653

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


The predicted value is: [0.87939835]
25
Entering main condition
1 month input [0.60082579 0.60631854 0.61141895 0.61687111 0.62616484 0.63591151
 0.64534377 0.65595307 0.66659111 0.676886   0.6929098  0.70859043
 0.72479375 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525
 0.91653199 0.94480878 0.97285677 1.         1.         0.87939835]
Pred_input shape is: (1, 24, 1)
1 month output [[0.8536481]]
Entering main condition
2 month input [0.60631854 0.61141895 0.61687111 0.62616484 0.63591151 0.64534377
 0.65595307 0.66659111 0.676886   0.6929098  0.70859043 0.72479375
 0.75504821 0.78279787 0.8135207  0.84738495 0.88252525 0.91653199
 0.94480878 0.97285677 1.         1.         0.87939835 0.85364813]
Pred_input shape is: (1, 24, 1)
2 month output [[0.8041145]]
Entering main condition
3 month input [0.61141895 0.61687111 0.62616484 0.63591151 0.64534377 0.65595307
 0.66659111 0.676886   0.6929098  0.70859043 0.72479375 0.75504821
 0.78279787 0.8135207  0.84738495 0.88252525 0.91653

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


IndexError: ignored

In [14]:
# After the new dataframe is completely filled with prediction data transform to CSV  
m_df.to_csv('completeHousePriceDataStPete.csv')

# Download final CSV file and it is ready to populate to DB
files.download('completeHousePriceDataStPete.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>